In [2]:
import tensorflow as tf
import os
import scipy.io
import numpy as np

2024-04-14 14:25:49.600737: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-14 14:25:49.680861: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-14 14:25:50.903314: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
working_dir = os.getcwd()

file_path = os.path.join(working_dir, 'jaxpi', 'examples', 'allen_cahn','data', 'allen_cahn.mat')

data = scipy.io.loadmat(file_path)

In [4]:
# get data for model
x = tf.convert_to_tensor(data['x'].flatten()[:,None])
t = tf.convert_to_tensor(data['t'].flatten()[:,None])

usol = (np.real(data['usol']))
usol = tf.reshape(usol, shape=(1,201,512))

In [5]:
print(len(x))

512


In [6]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(3, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(200, activation='tanh'),
    tf.keras.layers.Dense(1, name='output'),
])

model.compile(optimizer='adam', loss=tf.keras.losses.MeanSquaredError(), metrics=['accuracy'])

In [8]:

with tf.GradientTape(persistent=True) as tapex, tf.GradientTape(persistent=True) as tapet:
    tapex.reset()
    tapet.reset()

    tapex.watch(x)
    tapet.watch(t)

    inputs = tf.concat([t,x],axis=0)
    inputs = tf.transpose(inputs)

    
    model.fit(x=inputs,y=usol, epochs=50)

    # calculate gradients
    y = model(inputs)

    print(y)

    u_t = tapet.gradient(y, t, unconnected_gradients='zero')
    u_x = tapex.gradient(y, x, unconnected_gradients='zero')

    print(u_t)
    print(u_x)


Epoch 1/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 0.4559
Epoch 2/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 0.4559
Epoch 3/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.0000e+00 - loss: 0.4560
Epoch 4/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.0000e+00 - loss: 0.4562
Epoch 5/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0000e+00 - loss: 0.4562
Epoch 6/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 0.4562
Epoch 7/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step - accuracy: 0.0000e+00 - loss: 0.4560
Epoch 8/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.0000e+00 - loss: 0.4559
Epoch 9/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step - accuracy: 0.0000e+00 - loss: 0.4559
Epoch 10/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step - accuracy: 0.0000e+00 - loss: 0.4559
Epoch 11/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 40ms/step - accuracy: 0.0000e+00 - loss: 0.4560
Epoch 12/50
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 3

In [9]:
model.evaluate(x=inputs, y=usol)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step - accuracy: 0.0000e+00 - loss: 0.4559


[0.4558698832988739, 0.0]

In [250]:
# calculate gradient
with tf.GradientTape(persistent=True) as tape:
    tape.watch(x)
    forward = model(x)
    u_x = tape.gradient(forward, x, unconnected_gradients='zero')
    u_xx = tape.gradient(u_x, x, unconnected_gradients='zero')

#print(forward)
print(u_x)
print(u_xx)

tf.Tensor(
[[ 2.43167830e+00]
 [ 2.43249726e+00]
 [ 2.43330884e+00]
 [ 2.43411350e+00]
 [ 2.43491054e+00]
 [ 2.43570018e+00]
 [ 2.43648291e+00]
 [ 2.43725777e+00]
 [ 2.43802547e+00]
 [ 2.43878603e+00]
 [ 2.43953919e+00]
 [ 2.44028473e+00]
 [ 2.44102311e+00]
 [ 2.44175434e+00]
 [ 2.44247746e+00]
 [ 2.44319344e+00]
 [ 2.44390202e+00]
 [ 2.44460297e+00]
 [ 2.44529676e+00]
 [ 2.44598293e+00]
 [ 2.44666147e+00]
 [ 2.44733238e+00]
 [ 2.44799566e+00]
 [ 2.44865227e+00]
 [ 2.44930029e+00]
 [ 2.44994164e+00]
 [ 2.45057487e+00]
 [ 2.45123720e+00]
 [ 2.45185518e+00]
 [ 2.45246601e+00]
 [ 2.45306921e+00]
 [ 2.45366430e+00]
 [ 2.45425224e+00]
 [ 2.45483208e+00]
 [ 2.45540500e+00]
 [ 2.45596933e+00]
 [ 2.45652676e+00]
 [ 2.45707655e+00]
 [ 2.45761776e+00]
 [ 2.45815182e+00]
 [ 2.45867777e+00]
 [ 2.45919609e+00]
 [ 2.45970726e+00]
 [ 2.46021032e+00]
 [ 2.46070552e+00]
 [ 2.46119308e+00]
 [ 2.46167278e+00]
 [ 2.46214485e+00]
 [ 2.46260905e+00]
 [ 2.46306539e+00]
 [ 2.46351385e+00]
 [ 2.46395469e+00]
 

# advanced auto grad Examples

In [50]:
import tensorflow as tf

# Define your model
model = tf.keras.Sequential([
    tf.keras.Input(shape=(4,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(10, activation='relu', input_shape=(10,)),
    tf.keras.layers.Dense(1)
])

# Define some tensors
tensor1 = tf.constant([[1, 2], [3, 4]], dtype=tf.float32)
tensor2 = tf.constant([[5, 6], [7, 8]], dtype=tf.float32)

# Compute model output
with tf.GradientTape(persistent=True) as tape:
    tape.watch(tensor1)
    tape.watch(tensor2)

    # Concatenate the tensors along axis 0
    concatenated_tensor = tf.concat([tensor1, tensor2], axis=0)
    concatenated_tensor = tf.reshape(concatenated_tensor, (1,4,2))
    print(concatenated_tensor.shape)
    
    # Pass the concatenated tensor as input to the model
    model_output = model(concatenated_tensor)

# Compute gradients of the model output with respect to the individual tensors
gradient_tensor1 = tape.gradient(model_output, tensor1)
gradient_tensor2 = tape.gradient(model_output, tensor2)

print("Gradient of the model output with respect to tensor1:")
print(gradient_tensor1)

print("\nGradient of the model output with respect to tensor2:")
print(gradient_tensor2)


(1, 4, 2)
Gradient of the model output with respect to tensor1:
tf.Tensor(
[[-0.17236525  0.09680016]
 [-0.10308962 -0.23470856]], shape=(2, 2), dtype=float32)

Gradient of the model output with respect to tensor2:
tf.Tensor(
[[ 0.17332584  0.00868668]
 [-0.24331066  0.11009061]], shape=(2, 2), dtype=float32)
